# 案例: 分层控制系统

本脚本演示了一个两级分层控制系统，这是智能水务平台愿景的核心概念。该场景包含：

- **低层控制回路 (基于多智能体系统)**:
  - 一个 `DigitalTwinAgent` 发布水库的状态。
  - 一个 `LocalControlAgent` 订阅此状态，并使用PID控制器来控制一个水闸以达到设定的目标水位。
- **高层监控回路**:
  - 一个 `CentralDispatcher` 智能体订阅水库状态。
  - 基于一套规则（例如，如果水位过高），它会动态地为`LocalControlAgent`更改目标水位。

这展示了一个中央“大脑”如何根据整个系统的状态来管理局部控制器的目标。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# 导入平台组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher

CONFIG_PATH = 'examples/hierarchical_control.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义系统搭建函数

此函数将替代原 `helpers.py` 中的 `setup_hierarchical_control_system`，根据传入的配置和消息总线，搭建整个系统。

In [ ]:
def setup_system_from_config(cfg, message_bus):
    sim_cfg = cfg['simulation']
    topics = cfg['messaging_topics']
    comp_cfgs = cfg['components']
    agent_cfgs = cfg['agents']

    # 创建物理组件
    res_cfg = comp_cfgs['reservoir']
    reservoir = Reservoir("reservoir_1", res_cfg['initial_state'], res_cfg['params'])

    gate_cfg = comp_cfgs['gate']
    gate = Gate("gate_1", gate_cfg['initial_state'], gate_cfg['params'], message_bus, topics['gate_action'])
    
    # 创建低层智能体
    reservoir_twin = DigitalTwinAgent("twin_agent_reservoir_1", reservoir, message_bus, topics['reservoir_state'])
    gate_twin = DigitalTwinAgent("twin_agent_gate_1", gate, message_bus, topics['gate_state'])

    pid_cfg = agent_cfgs['pid_controller']
    pid_controller = PIDController(setpoint=pid_cfg['setpoint'], Kp=pid_cfg['Kp'], Ki=pid_cfg['Ki'], Kd=pid_cfg['Kd'], min_output=0.0, max_output=gate_cfg['params']['max_opening'])

    control_agent = LocalControlAgent(
        "control_agent_gate_1", pid_controller, message_bus, 
        observation_topic=topics['reservoir_state'], observation_key='water_level',
        action_topic=topics['gate_action'], dt=sim_cfg['dt'], 
        command_topic=topics['gate_command'], feedback_topic=topics['gate_state']
    )

    # 创建高层智能体
    disp_cfg = agent_cfgs['dispatcher']
    dispatcher = CentralDispatcher(
        "central_dispatcher_1", message_bus, 
        state_subscriptions={'reservoir_level': topics['reservoir_state']},
        command_topics={'gate1_command': topics['gate_command']},
        rules=disp_cfg['rules']
    )

    components = [reservoir, gate]
    agents = [reservoir_twin, gate_twin, control_agent, dispatcher]
    
    print("系统组件和智能体已根据配置创建完毕。")
    return components, agents

### 3. 运行仿真并可视化

In [ ]:
# 1. 创建仿真平台和消息总线
harness = SimulationHarness(config=config['simulation'])
message_bus = harness.message_bus

# 2. 使用配置搭建系统
components, agents = setup_system_from_config(config, message_bus)

# 3. 将组件和智能体添加到仿真平台
for component in components:
    harness.add_component(component)
for agent in agents:
    harness.add_agent(agent)

# 4. 定义拓扑并构建
harness.add_connection("reservoir_1", "gate_1")
harness.build()

# 5. 运行仿真
print("\n--- 开始运行仿真 ---")
harness.run_mas_simulation()
print("--- 仿真结束 ---")

# 6. 提取并处理结果
history = harness.history
flat_history = []
for step in history:
    row = {'time': step['time']}
    for component_name, states in step.items():
        if component_name == 'time' or not isinstance(states, dict): continue
        for state_key, value in states.items():
            row[f"{component_name}.{state_key}"] = value
    flat_history.append(row)
results_df = pd.DataFrame(flat_history).set_index('time')

# 7. 绘制结果
fig, ax1 = plt.subplots(figsize=(15, 7))
fig.suptitle('分层控制仿真结果', fontsize=16)

ax1.plot(results_df.index, results_df['reservoir_1.water_level'], 'b-', label='水库水位 (m)')
ax1.set_xlabel('时间 (s)')
ax1.set_ylabel('水位 (m)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True)
ax1.axhline(y=config['agents']['dispatcher']['rules']['flood_threshold'], color='r', linestyle=':', label='洪水阈值')

ax2 = ax1.twinx()
ax2.plot(results_df.index, results_df['gate_1.opening'], 'g-', label='水闸开度 (m)')
ax2.set_ylabel('开度 (m)', color='g')
ax2.tick_params(axis='y', labelcolor='g')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.show()